In [1]:
import sqlite3
import timeit
import pandas as pd
import datetime as dt
from datetime import timezone
import aisCore

In [2]:
# Get data from database
database = 'c:/aisdata.db'
con = sqlite3.connect(database)
cur = con.cursor()
cur.execute('SELECT name from sqlite_master where type= "table"')
print(cur.fetchall())

In [5]:
fromDate = dt.datetime(2018,4,1)
toDate = dt.datetime(2018,7,1)

In [6]:
def extractAISData(mmsi, fromDate, toDate):
    
    # Connect to the database, and get cursor
    database = 'c:/aisdata.db'
    con = sqlite3.connect(database)
    cur = con.cursor()
    
    # Convert the datetime objects to utc timestamps
    utc_from = int(fromDate.replace(tzinfo=timezone.utc).timestamp())
    utc_to = int(toDate.replace(tzinfo=timezone.utc).timestamp())

    # Create SQL statements
    sqlPos = "SELECT * FROM messagetype1 WHERE mmsi = {} AND unixtime BETWEEN {} AND {}".format(mmsi, utc_from, utc_to)
    
    # Read relevant data from database
    dfPos = pd.read_sql_query(sqlPos, con)
    #dfShips = pd.read_sql_query("SELECT * FROM messagetype5 WHERE unixtime BETWEEN 1517270400 AND 1517356800", con)

    return dfPos

In [14]:
tStart = dt.datetime.now()
mmsiEmmaMaersk = 220417000
#mmsi = 538005505 #21884900
dfPos = extractAISData(mmsiEmmaMaersk, fromDate, toDate)
tEnd = dt.datetime.now()
(tEnd-tStart).seconds

241

In [15]:
dfPos

,mmsi,unixtime,latitude,longitude,heading,sog,nav_status,cog,rot
0,220417000,1522542117,5.748685,80.259188,110,18.799999,0,110.599998,0.000000
1,220417000,1522543864,5.710098,80.401925,95,18.100000,0,95.300003,1.116007
2,220417000,1522551963,5.715958,81.069348,88,17.200001,0,88.400002,-1.116007
3,220417000,1522551963,5.715958,81.069348,88,17.200001,0,88.400002,-1.116007
4,220417000,1522553936,5.720725,81.226253,88,17.100000,0,88.199997,-2.856978
...,...,...,...,...,...,...,...,...,...
1459,220417000,1529429764,1.203993,103.583935,58,10.700000,0,52.200001,0.000000
1460,220417000,1529540509,1.219175,103.921575,72,14.400000,0,71.300003,0.000000
1461,220417000,1529541003,1.229207,103.954415,73,15.100000,0,73.199997,0.000000
1462,220417000,1529658078,6.611833,107.486032,25,12.600000,0,26.500000,0.000000


In [27]:
vEmma = Vessel(220417000, 9321483, 'OYGR2','Emma Mærsk')

In [28]:
for row in dfPos.iterrows():
    vEmma.addObs(row[1]['unixtime'],row[1]['latitude'],row[1]['longitude'])

In [36]:
import pickle

fnamePickle = 'routedata_emma_maersk'
f = open(fnamePickle,'wb')
pickle.dump(vEmma,f)
f.close()


In [3]:
import pickle

fnamePickle = 'routedata_emma_maersk'

with open(fnamePickle, 'rb') as f:
    vEmma2 = pickle.load(f)

In [4]:
vEmma2.getRoute()

,dt,lat,lon
0,2018-04-01 00:21:57,5.748685,80.259188
1,2018-04-01 00:51:04,5.710098,80.401925
2,2018-04-01 03:06:03,5.715958,81.069348
3,2018-04-01 03:38:56,5.720725,81.226253
4,2018-04-01 04:12:40,5.724055,81.386320
...,...,...,...
1364,2018-06-19 16:30:48,1.224573,103.357000
1365,2018-06-19 17:36:04,1.203993,103.583935
1366,2018-06-21 00:21:49,1.219175,103.921575
1367,2018-06-21 00:30:03,1.229207,103.954415


In [5]:
# Import mapbox to show positions in a map
import plotly.express as px
import plotly.graph_objects as go
px.set_mapbox_access_token('pk.eyJ1Ijoic3RlaW5vdmVlcmlrc3RhZCIsImEiOiJjazlzeDJ2YmwxNnA0M2ltb3d0dDRiem83In0.dZch3BJHwwxJYGlN0pCSWg')

In [8]:
# Show positions of all the vessels in a map 
fig = px.scatter_mapbox(vEmma2.getRoute(), lat="lat", lon="lon") #,center=go.layout.mapbox.Center(lat=60,lon=5), zoom=4)
fig.update_layout(
    title='Ship worldwide positions 2018-02-28',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        bearing=0,
        center=dict(lat=50,lon=5),
        pitch=0,
        zoom=1,
        style='light'
    ),
)
fig.show()

In [2]:
class Vessel:
    def __init__(self, mmsi, imo, callsign, name):
        self.mmsi = mmsi
        self.imo = imo
        self.callsign = callsign
        self.name = name
        
        self.route = {}
        
    def addObs(self,t,lat,lon):
        dtObs = dt.datetime.utcfromtimestamp(t).strftime("%Y-%m-%d %H:%M:%S")
        self.route[t] = (dtObs,lat,lon)
        
    def getRoute(self):
        lst = [self.route[key] for key in self.route]
        df = pd.DataFrame(lst,columns=['dt','lat','lon'])
        return df
    
    def getFirstPosOnRoute(self):
        return list(self.route.values())[0]
    
    def __repr__(self):
        return 'Vessel: MMSI %s, Name: %s, IMO: %s, Callsign: %s' % (str(self.mmsi), str(self.name), str(self.imo), str(self.callsign))


class Fleet:
    def __init__(self, name):
        self.name = name
        self.vessels = {}
    
    def getFirstPos(self):
        lst = [v.getFirstPosOnRoute() for v in list(self.vessels.values())]
        dfRet = pd.DataFrame(lst, columns=['dt','lat','lon'])
        dfRet['name'] = [v.name for v in list(self.vessels.values())]
        return dfRet